In [11]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt

In [28]:
dataset = pd.read_csv('Restaurant_Reviews.tsv', delimiter = '\t', quoting = 3) # Quoting 3 ignora as aspas
dataset.isnull().sum()

Review    0
Liked     0
dtype: int64

In [38]:
import re 
import nltk 
nltk.download('stopwords') # Stopwords são palavras muito comuns em um idioma, como "o", "a", "e", "de", "para" em português, ou "the", "and", "is", "in" em inglês.
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

corpus = []
for i in range(0, len(dataset)):
    review = re.sub('[^a-zA-Z]', ' ', dataset['review'][i])
    review = review.lower()
    review = review.split()


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Gabriel\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
